# Interact with API

For easier use, we have deployed endpoints which can be used to test the models. The s3 link of model and files you want to test are needed. 



In [ ]:
!pip install rasterio folium matplotlib numpy requests boto3 -q

## Restart Kernel
Kernel > Restart Kernel

## Setup
1. Navigate to https://nasa-impact.awsapps.com/start
2. Click on `Amazon Sagemaker Studio`
3. Click on `Open Launcher`
4. Click on `System terminal`
5. Once the terminal starts, run `git clone https://github.com/NASA-IMPACT/2023-igarss-tutorial.git`
6. Click on the folder icon on the left control pane
7. Double Click on `2023-igarss-tutorial`
8. Double Click on `chapter-2`
9. Double Click on `chapter-2.ipynb`
10. Leave default values for the environment and click `Select`


## Accessing s3 link of uploaded model
1. Navigate to https://nasa-impact.awsapps.com/start
2. Login
3. Click on `AWS Account`
4. Click on `Summer School`
5. Click on `Management Console`
6. In the search bar, search for `s3`
7. Click on `s3`
8. Click on `2023-igarss-tutorial-data`
9. Click on your `username`
10. Click on `<username>_<experiment>.pth`. Replace `<username>` with your username, and `<experiment>` with either `burn_scars` or `flood`.
11. Copy S3 URI (Eg: s3://2023-igarss-tutorial-store/models/gurung1/gurung1_burn_scars.pth)


In [ ]:
# update with copied s3 link
model_path = <your model path>

## Accessing s3 link of test files
1. Navigate to https://nasa-impact.awsapps.com/start
2. Login
3. Click on `AWS Account`
4. Click on `Summer School`
5. Click on `Management Console`
6. In the search bar, search for `s3`
7. Click on `s3`
8. Click on `2023-igarss-tutorial-data`
9. Click on `data`
10. Click on `burn_scars`
11. Click on any file that ends on `_merged.tif`
12. Copy S3 URI (Eg: s3://2023-igarss-tutorial-store/data/burn_scars/subsetted_512x512_HLS.S30.T10SFF.2020215.v1.4_merged.tif)


In [ ]:
def convert_rgb(img):
    """
    Convert passed list of bands to proper RGB image for visualization purposes
    Args:
        img: numpy array of GOES data (6 bands)
    """
    red = img[:, :, 1].astype('uint8')
    blue = img[:, :, 0].astype('uint8')
    pseudo_green = img[:, :, 2].astype('uint8')
    height, width = red.shape

    img = np.moveaxis(
        np.array([red, pseudo_green, blue]), 0, -1
    )

    return img

def visualize(filename, predictions):
    """
    Method to visualize the results alongside the test data
    ARGs:
       filename: Name of the test file (local path) 
       predictions: response from the `infer` method for the corresponding file name.
    """
    preds = np.asarray(predictions) * 255
    raster = rasterio.open(filename).read()
    img = convert_rgb(raster)
    plt.imshow(img, preds[0])
    raster.close()

In [ ]:
## Infering from the API

# update this for visualization. may be use maps?

import boto3
import io 
import numpy.ma as ma

session = boto3.session.Session()

s3_client = session.client('s3')

arr = rasterio.open(file_path).read()
arr = np.moveaxis(arr, 0, -1)
f, ax = plt.subplots(1, 2)
ax[0].imshow(convert_rgb(arr).astype('uint8'))
ax[1].imshow(convert_rgb(arr).astype('uint8'))
predictions = rasterio
ax[1].imshow(
    ma.masked_where(
        predictions < 50, predictions
    ),
    alpha=0.95,
    cmap='Oranges'
)
plt.show()

In [ ]:
# update with copied s3 link
file_path = <your file path>

In [ ]:
import requests
import json

url = "http://hls-foundation.nasa-impact.net/models/burn_scars/infer"

payload = json.dumps({
  "model_path": model_path,
  "file_path": file_path
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


## Visualizing the predicitons

In [ ]:
!pip install rasterio folium

In [ ]:
import folium
import folium.plugins as plugins
import rasterio
from rasterio.plot import show
import numpy as np


def colorize(array, cmax, cmin=0, cmap="rainbow"):
    """Converts a 2D numpy array of values into an RGBA array given a colour map and range.
    Args:
        array (ndarray):
        cmax (float): Max value for colour range
        cmin (float): Min value for colour range
        cmap (string): Colour map to use (from matplotlib colourmaps)
    Returns:
            rgba_array (ndarray): 3D RGBA array which can be plotted.
    """
    normed_data = (array - cmin) / (array.max() - cmin)
    cm = plt.cm.get_cmap(cmap)
    return cm(normed_data)



In [ ]:
inference_files = sorted(glob.glob('/opt/app-root/src/data/' + project_name + '/inference/*.tif'))
inference_files

In [ ]:
for filenum in range(len(inference_files)):
    original_file = inference_files[filenum]
    predict_file = inference_files[filenum].replace('/inference','/inference/pred/' + experiment_name).replace('.tif','_pred.tif')
    # Load the original image layer
    with rasterio.open(original_file) as src:
        redArray = src.read(1)
        greenArray = src.read(2)
        blueArray = src.read(3)
        bounds = src.bounds
        nd = src.nodata
        midLat = (bounds[3] + bounds[1]) / 2
        midLon = (bounds[2] + bounds[0]) / 2
        im_rgb = np.moveaxis(np.array([redArray,greenArray,blueArray]), 0, -1)
        im_rgb = im_rgb/np.max(im_rgb)
    # Create the map
    if filenum == 0:
        m = folium.Map(location=[midLat, midLon], tiles='openstreetmap', max_zoom=22)
    # Add the prediciton layer to the map
    with rasterio.open(predict_file) as src:
        dataArray = src.read(1)
        bounds = src.bounds
        nd = src.nodata
    # cmax = np.max(dataArray)
    cmax = 1000
    dataArrayMasked = np.ma.masked_where(dataArray == nd, dataArray)
    dataArrayMasked = np.ma.masked_where(dataArray == 0, dataArrayMasked)
    imc = colorize(dataArrayMasked, cmax, cmin=0, cmap="viridis")
    # Add the layers to the map
    pred = folium.raster_layers.ImageOverlay(imc, [[bounds[1], bounds[0]], [bounds[3], bounds[2]]], name=f"Prediction-{filenum}", opacity=0.8)
    orig = folium.raster_layers.ImageOverlay(im_rgb, [[bounds[1], bounds[0]], [bounds[3], bounds[2]]], name=f"Original image{filenum}", opacity=1.0)
    orig.add_to(m)
    pred.add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds(bounds)
m    